In [3]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp


import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import synthetic.tools as tools
import synthetic.emulator.emulator as emulator
import synthetic.emulator.indexer as indexer

In [5]:
tag_root = "dc2-alpha_concentric_sample-v01_test-03"
NREPEATS = 4
NSAMPLES = 1600000
NCHUNKS = 160
bandwidth=0.1

root_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/"
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

LOGR_DRAW_RMINS = np.array([-3, -0.5, 0., 0.5])
LOGR_DRAW_RMAXS = np.array([-0.5, 0., 0.5, 1.2])
LOGR_CAT_RMAXS = [0., 0.5, 1.1, 1.2]

deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass")        
    ],
    "logs": [False, True, False, False, False, False, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16)],
}

wide_cr_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass"),
        ("LOGR", "R"),        
    ],
    "logs": [False, True, False, False, False, False, True, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16), (1e-3, 16)],
}

columns = {
    "cols_dc": ["COLOR_G_R", "COLOR_R_I",],
    "cols_wr": ["LOGR",],
    "cols_wcr": ["COLOR_G_R", "COLOR_R_I", "LOGR",],
}


In [6]:
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

In [ ]:
refpixel = pd.read_hdf(deep_data_path, key="data")
table = pd.read_hdf(wide_data_path, key="data")

In [ ]:
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_rands = emulator.construct_deep_container(refpixel, tmp_wide_cr_settings)

# loading deep catalogs
_deep_smc_settings = emulator.construct_deep_container(refpixel, deep_smc_settings)

# loading cluster data
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_clust = emulator.construct_deep_container(table, tmp_wide_cr_settings)
